In this Jupyter notebook, we will output the datasheets for the algo project. 

First, let's work on the bh sheet which shows all trades within the program and shows all dates including the ones when the signal did not generate an order. 


In [11]:

#Imports 
import yfinance as yf
import pandas as pd 
import numpy as np 
import warnings 



# Ignore all warnings
warnings.filterwarnings('ignore')

#Import Stock Data from Yfinance

# Get the TQQQ stock data from July 1st, 2013 to July 1st, 2023 using YFinance
data = yf.download("TQQQ", start="2020-07-06", end="2023-07-01")
# Round columns to two places after decimal. 
df = pd.DataFrame(data).round(2)


# Drop Adj Close 
df = df.drop(["Adj Close"], axis=1)

# Add Tradeno column to the Data and DataFrame
Tradeno = len(df.index)
df['Tradeno']= Tradeno
# Add a Column bhTradeno, Symbol, and Set bhDate to Index Column
df["Tradeno"] = range(1, len(df) + 1)
df['Date'] = df.index
# Set Tradeno as the index
df = df.set_index("Tradeno")
""""
# Common Definitions for both Buy + Hold(BH) and Strategy (Strat)
Shares
Symbol
Initial Balance
Range 
Delta 
"""
# Add Shares Column 
df['Shares'] = 1000
Shares = df['Shares']
Shares = 1000

# Add Symbol column
Symbol = ["TQQQ"] * (len(df.index))
df['Symbol'] = Symbol



# ***Please note that for Starting Balances, we used 26830 so that both strategies are 
# *** using the same amount based on the BH strat first purchase on first trade date. 
# *** Our backtest strategy starts with the same amount on the 4th day of the observation
# *** BH has a small lead over the Backtest Strategy due to the 4 day advantage. 

# Add Initial Balance
IB = 26830



# Add Range Column (distance from daily low to high)
df['Range']= df['High'] - df['Low']
Range = df['Range']

# Add Change Column (change from open price to close price)
Change = df['Close'] - df['Open']
df['Change'] = Change 

"""
Begin setting definitions for Strategy 
"""
# Set the parameters for the strategy
short_window = 2
long_window = 10

# Calculate the short and long moving averages
df['short_mavg'] = df['Close'].rolling(window=short_window, min_periods=1, center=False).mean().round(2)
df['long_mavg'] = df['Close'].rolling(window=long_window, min_periods=1, center=False).mean().round(2)

# Create a 'Signal' column indicating when the short moving average crosses the long.
df['Signal'] = np.where(df['short_mavg'] > df['long_mavg'], 1.0, 0.0)

# Create a 'Position' column indicating whether to be long or short the stock
df['Position'] = df['Signal'].diff()

# Create a new column named "Order", buy the day after (diff) when signal is generated, buy on the open, sell at the close! 
df["StratOrder"] = ""
StratOrder = df['StratOrder']
df.loc[df["Position"] == 1.0, "StratOrder"] = "Buy"
df.loc[1.0, "Position"] = "0"

# Create bhBalance from shares * close 

bhBalance = Shares * df['Close']
df['bhBalance']= bhBalance

# Calculate the bh drawdown
drawdown = df["bhBalance"].max() - df["bhBalance"]
df['bhDrawdown'] = drawdown



# Buy and Hold Strategy Pct Return and Cumulative Strategy Pct Return 

# Define initial balance and strategy balance
initial_balance = IB  # Example initial balance
bhbalance = df['bhBalance']    # Example strategy balance

# Calculate strategy return
bhreturn = ((bhbalance - initial_balance) / initial_balance) 
bhreturnpct= ((bhbalance - initial_balance) / initial_balance) * 100


# Return is cumulative for bh strategy (buy and hold strategy)
df['bhreturn']=bhreturn
df['bhreturnpct']=bhreturnpct 

df = df.round(2)


# Rearrange Column Order 
column_order = ['Date', 'Shares', 'Symbol', 'StratOrder','Open', 'High', 'Low', 'Close', 'Change', 'Range','Volume', 'bhBalance', 'bhDrawdown', 'bhreturn','bhreturnpct']
df = df[column_order]

display(df)


[*********************100%***********************]  1 of 1 completed


,Date,Shares,Symbol,StratOrder,Open,High,Low,Close,Change,Range,Volume,bhBalance,bhDrawdown,bhreturn,bhreturnpct
Tradeno,,,,,,,,,,,,,,,
1,2020-07-06,1000,TQQQ,,26.83,27.90,26.82,27.72,0.89,1.08,105992000,27720.0,60850.0,0.03,3.32
2,2020-07-07,1000,TQQQ,,27.64,28.54,27.06,27.17,-0.47,1.48,123049600,27170.0,61400.0,0.01,1.27
3,2020-07-08,1000,TQQQ,,27.61,28.24,27.12,28.22,0.61,1.12,119965600,28220.0,60350.0,0.05,5.18
4,2020-07-09,1000,TQQQ,Buy,28.85,29.17,27.47,28.92,0.07,1.70,158798000,28920.0,59650.0,0.08,7.79
5,2020-07-10,1000,TQQQ,,28.85,29.61,27.97,29.54,0.69,1.64,121394800,29540.0,59030.0,0.10,10.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,2023-06-26,1000,TQQQ,,38.76,39.67,37.34,37.41,-1.35,2.33,108450000,37410.0,51160.0,0.39,39.43
750,2023-06-27,1000,TQQQ,,37.85,39.50,37.60,39.26,1.41,1.90,97677000,39260.0,49310.0,0.46,46.33
751,2023-06-28,1000,TQQQ,,38.67,40.14,38.63,39.50,0.83,1.51,108676300,39500.0,49070.0,0.47,47.22


Below, we create the sdf dataframe which introduces the backtest strategy results and calculations.  

In [12]:

#### We can now create a dataframe by pivoting to only the Buy orders for the strategy.


# Begin definitions 
stratorders_df = df[df['StratOrder'] == 'Buy']
sdf = stratorders_df 
sdf['StratOrder'] = df['StratOrder']
# Define and Calculate Profits
sdf['Strat_Profits'] = Change * Shares
Strat_Profits = sdf['Strat_Profits']
# Define and Calculate Balance and Strat Initial Balance (the price at the open of first day trade)
SIB = 26830
cum_profit = sdf['Strat_Profits'].cumsum()
sdf['Strat_Balance'] = cum_profit  + SIB 
Strat_Balance = sdf['Strat_Balance']

# Win Loss Column added
def is_profitable(row):
    if row['Close'] > row['Open']:
        return 'Win'
    else:
        return 'Loss'

sdf['WinLoss'] = sdf.apply(is_profitable, axis=1)

# Add 'WIN' column to store positive profits
def get_win_amount(row):
    if row['Strat_Profits'] > 0:
        return row['Strat_Profits']
    else:
        return 0.00

sdf['Win'] = sdf.apply(get_win_amount, axis=1)

# Add 'Loss' column to store negative profits
def get_loss_amount(row):
    if row['Strat_Profits'] < 0:
        return row['Strat_Profits']
    else:
        return 0.00

sdf['Loss'] = sdf.apply(get_loss_amount, axis=1)

# Strategy Returns Column 
sdf['Strat_Returns_pct'] =((Strat_Balance/IB) -1)*100
Strat_Returns = sdf['Strat_Returns_pct']

# Calculate the bh drawdown
drawdown = sdf["bhBalance"].max() - sdf["bhBalance"]
sdf['bhDrawdown'] = drawdown

#calculate strategy drawdown 
stratdrawdown = sdf['Strat_Balance'].max() - sdf['Strat_Balance']
sdf['Strat_Drawdown']= stratdrawdown 

# Define initial balance and strategy balance
initial_balance = SIB  # Example initial balance
strat_balance = sdf['Strat_Balance']    # Example strategy balance

# Calculate strategy return
strategy_return = ((strat_balance - initial_balance) / initial_balance) 
stratreturnpct = ((strat_balance - initial_balance) / initial_balance) * 100
sdf['strategy_return']= strategy_return
sdf['stratreturnpct']= stratreturnpct


# Define initial balance and strategy balance
initial_balance = 27000  # Example initial balance
bhbalance = sdf['bhBalance']    # Example strategy balance

# Calculate strategy return
bhreturn = ((bhbalance - initial_balance) / initial_balance) 
bhreturnpct= ((bhbalance - initial_balance) / initial_balance) * 100
sdf['bhreturn']=bhreturn
sdf['bhreturnpct']=bhreturnpct 

# Rearrange Column Order 
column_order = ['Date', 'Shares', 'Symbol', 'Open', 'High', 'Low', 'Close','Volume','Range', 'Change','StratOrder', 'bhBalance','Strat_Balance', 'Strat_Profits', 'WinLoss', 'Win', 'Loss','Strat_Drawdown', 'bhDrawdown','bhreturn', 'bhreturnpct','strategy_return', 'stratreturnpct']
sdf = sdf[column_order]

display(sdf)


,Date,Shares,Symbol,Open,High,Low,Close,Volume,Range,Change,...,Strat_Profits,WinLoss,Win,Loss,Strat_Drawdown,bhDrawdown,bhreturn,bhreturnpct,strategy_return,stratreturnpct
Tradeno,,,,,,,,,,,,,,,,,,,,,
4,2020-07-09,1000,TQQQ,28.85,29.17,27.47,28.92,158798000,1.70,0.07,...,70.0,Win,70.0,0.0,46650.0,55970.0,0.071111,7.111111,0.002609,0.260902
8,2020-07-15,1000,TQQQ,28.72,29.06,27.35,28.44,183772000,1.71,-0.28,...,-280.0,Loss,0.0,-280.0,46930.0,56450.0,0.053333,5.333333,-0.007827,-0.782706
11,2020-07-20,1000,TQQQ,28.20,30.56,27.77,30.34,125810800,2.79,2.14,...,2140.0,Win,2140.0,0.0,44790.0,54550.0,0.123704,12.370370,0.071934,7.193440
20,2020-07-31,1000,TQQQ,29.83,29.88,28.22,29.84,179907600,1.66,0.01,...,10.0,Win,10.0,0.0,44780.0,55050.0,0.105185,10.518519,0.072307,7.230712
29,2020-08-13,1000,TQQQ,32.02,32.79,31.63,31.98,117140800,1.16,-0.04,...,-40.0,Loss,0.0,-40.0,44820.0,52910.0,0.184444,18.444444,0.070816,7.081625
60,2020-09-28,1000,TQQQ,32.32,32.50,31.39,32.43,136665600,1.11,0.11,...,110.0,Win,110.0,0.0,44710.0,52460.0,0.201111,20.111111,0.074916,7.491614
87,2020-11-04,1000,TQQQ,33.90,35.66,33.33,35.02,169187600,2.33,1.12,...,1120.0,Win,1120.0,0.0,43590.0,49870.0,0.297037,29.703704,0.116660,11.666045
115,2020-12-15,1000,TQQQ,42.15,42.50,41.23,42.47,78708800,1.27,0.32,...,320.0,Win,320.0,0.0,43270.0,42420.0,0.572963,57.296296,0.128587,12.858740
131,2021-01-08,1000,TQQQ,46.83,47.72,45.85,47.57,76407600,1.87,0.74,...,740.0,Win,740.0,0.0,42530.0,37320.0,0.761852,76.185185,0.156168,15.616847


Below, the new dataframe, ndf, cleans the data so we can compare balances on the benchmark, buy and hold against the backtest strategy.  

In [13]:


neworder = ['Date', 'Shares', 'Symbol',  'Open', 'Close','Change','StratOrder', 'bhBalance','Strat_Balance', 'Strat_Profits']
ndf = sdf[neworder]
print(ndf)
#display(ndf.tail(5))

              Date  Shares Symbol   Open  Close  Change StratOrder  bhBalance  \
Tradeno                                                                         
4       2020-07-09    1000   TQQQ  28.85  28.92    0.07        Buy    28920.0   
8       2020-07-15    1000   TQQQ  28.72  28.44   -0.28        Buy    28440.0   
11      2020-07-20    1000   TQQQ  28.20  30.34    2.14        Buy    30340.0   
20      2020-07-31    1000   TQQQ  29.83  29.84    0.01        Buy    29840.0   
29      2020-08-13    1000   TQQQ  32.02  31.98   -0.04        Buy    31980.0   
60      2020-09-28    1000   TQQQ  32.32  32.43    0.11        Buy    32430.0   
87      2020-11-04    1000   TQQQ  33.90  35.02    1.12        Buy    35020.0   
115     2020-12-15    1000   TQQQ  42.15  42.47    0.32        Buy    42470.0   
131     2021-01-08    1000   TQQQ  46.83  47.57    0.74        Buy    47570.0   
138     2021-01-20    1000   TQQQ  47.93  49.49    1.56        Buy    49490.0   
147     2021-02-02    1000  

In [14]:
# Print the df DataFrame to a CSV file
df.to_csv("df.csv")
# Print the Backtest Strategy DataFrame to a CSV file
sdf.to_csv("sdf.csv")